In [3]:
import tensorquant as tq
from datetime import date 
import matplotlib.pyplot as plt
import pandas as pd 
# import tensorflow as tf
# import tensorflow_probability as tfp
import numpy as np

# Settings

In [5]:
tq.Settings.evaluation_date = date(2024, 7, 31)
trade_date = tq.Settings.evaluation_date

# Market

In [9]:
market = {}
daycounter = tq.DayCounter(tq.DayCounterConvention.ActualActual)

curves_df = pd.read_excel("data/market_20240731.xlsx", sheet_name='curves')
curves_df['start'] = curves_df['start'].dt.date
curves_df['end'] = curves_df['end'].dt.date
estr_df = curves_df[curves_df['name'] == 'EUR_ESTR']
eur6m_df = curves_df[curves_df['name'] == 'EUR_6M']

rates = estr_df['quote'].values/100
#tq
curve_tq = tq.RateCurve(reference_date=tq.Settings.evaluation_date,
                         pillars=estr_df['end'],
                         rates=rates,
                         interp='LINEAR',
                         daycounter_convention=tq.DayCounterConvention.ActualActual)

vol_df = pd.read_excel("data/market_20240731.xlsx", sheet_name = 'vols', index_col=0)
default_vol = vol_df.loc['DEFAULT']
maturities = default_vol['tenor'].unique()
strikes = default_vol['strike'].unique()

vol_matr = np.zeros((len(maturities), len(strikes)))
for i, t in enumerate(maturities):
    vol_matr[i, :] = default_vol[default_vol['tenor'] == t]['quote'].values
    
surface = tq.VolatilitySurface(reference_date=tq.Settings.evaluation_date,
                            calendar=tq.TARGET(),
                            daycounter=tq.DayCounter(tq.DayCounterConvention.Actual365),
                            strike=strikes,
                            maturity=maturities/365,
                            volatility_matrix=vol_matr)

spot_df = pd.read_excel("data/market_20240731.xlsx", sheet_name = 'spot', index_col=0)

market['IR:EUR:ESTR'] = curve_tq
market['VOLEQ:DEFAULT'] = surface
market['EQ:DEFAULT'] = spot_df.loc['DEFAULT']['quote']

# Option

# TQ

### BSM

In [10]:
maturity_date = tq.TARGET().advance(tq.Settings.evaluation_date, 6, tq.TimeUnit.Months, tq.BusinessDayConvention.ModifiedFollowing)
strike_price = 130

option = tq.VanillaOption(tq.Currency.EUR, tq.Settings.evaluation_date, maturity_date, tq.OptionType.Call,
                             underlying = 'DEFAULT',strike = strike_price)
black_pricer = tq.BlackScholesPricer(tq.market_map)
black_pricer.price(option, market, True)

In [11]:
option.price

<tf.Tensor: shape=(), dtype=float32, numpy=11.509682>

In [12]:
black_pricer.tape.gradient(option.price, [black_pricer._s, black_pricer._r, black_pricer._sigma, black_pricer._t])


[<tf.Tensor: shape=(), dtype=float32, numpy=0.5430066>,
 <tf.Tensor: shape=(), dtype=float32, numpy=29.131895>,
 <tf.Tensor: shape=(), dtype=float32, numpy=35.938335>,
 <tf.Tensor: shape=(), dtype=float32, numpy=13.412939>]

### MC

In [13]:
s0 = black_pricer._s.numpy()
r = black_pricer._r.numpy()
sigma = black_pricer._sigma.numpy()
t = black_pricer._t.numpy()
gbm = tq.GeometricBrownianMotion(r,sigma, s0)

In [14]:
import tensorflow as tf
n_path = 200000
timesteps = 200
z = tf.random.normal((n_path, timesteps), seed=12, dtype=tf.dtypes.float64)

In [15]:
t = tf.Variable(t, dtype=tf.float64)
with tf.GradientTape() as tape:
    s_t = gbm.evolve(t, z)
    payoff = tf.math.maximum(s_t[:, -1] - strike_price, 0)
    p_mc = tf.exp(-gbm._mu * t) * tf.reduce_mean(payoff)

sensy = tape.gradient(p_mc, [gbm._x0, gbm._mu, gbm._sigma, t])
print("Black AAD: ")
print("Price: ", p_mc.numpy())
print("*"*30)
print(f"Delta: {sensy[0].numpy()}")
print(f"Rho: {sensy[1].numpy()}")
print(f"Vega: {sensy[2].numpy()}")
print(f"Theta: {sensy[3].numpy()}")
print("*"*30)

Black AAD: 
Price:  11.549910803860488
******************************
Delta: 0.542190725790068
Rho: 29.059129051217027
Vega: 36.12936387712836
Theta: 13.46892131636077
******************************
